<h3> Base de dados de crédito </h3>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [29]:
df = pd.read_csv('credit-data.csv', sep=',')

In [3]:
list(df)

['clientid', 'income', 'age', 'loan', 'default']

In [4]:
df.sample()

,clientid,income,age,loan,default
1098,1099,41435.153752,34.925716,3702.171428,0


In [5]:
df.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [7]:
df.loc[df['age']<0]

,clientid,income,age,loan,default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


In [8]:
df.drop(df[df.age < 0].index, inplace = True)

In [9]:
df['age'][df.age > 0].mean()

40.92770044906149

In [30]:
#Substituir os valores errados com idade negativa pela média
df.loc[df.age <0] = df['age'][df.age > 0].mean()

In [11]:
#Há valofes nulos?
df.isnull().values.any()

True

In [12]:
#Há valores nulos na idade
df.age.isnull().values.any()

True

In [13]:
#Quantos valores nulos há em cada coluna
df.isnull().sum(axis = 0)

clientid    0
income      0
age         3
loan        0
default     0
dtype: int64

In [14]:
df.loc[pd.isnull(df['age'])] = df['age'][df.age > 0].mean()

<h3> Separação do dataset em dois conjuntos </h3>

In [15]:
df.head()

,clientid,income,age,loan,default
0,1.0,66155.925095,59.017015,8106.532131,0.0
1,2.0,34415.153966,48.117153,6564.745018,0.0
2,3.0,57317.170063,63.108049,8020.953296,0.0
3,4.0,42709.534201,45.751972,6103.642260,0.0
4,5.0,66952.688845,18.584336,8770.099235,1.0


In [16]:
x_all = df.iloc[:,1:4].values

In [111]:
xdf = df.drop(['default'], axis = 1)
ydf = df['default'].values

In [100]:
y_all = df.iloc[:,4].values

<h3> Pré-processar os valores missing </h3>

In [101]:
from sklearn.preprocessing import Imputer

In [102]:
#Como mostrar apenas os valores missing de alguma feature
df.loc[pd.isnull(df['age'])]

,clientid,income,age,loan,default
28,29.0,59417.805406,NaN,2082.625938,0.0
30,31.0,48528.852796,NaN,6155.784670,0.0
31,32.0,23526.302555,NaN,2862.010139,0.0


In [103]:
imputer = Imputer(missing_values="NaN", strategy = 'mean', axis = 0)

In [104]:
xarray = np.array(xdf)
yarray = np.array(ydf)

In [105]:
#Só vou aplicar o imputer sobre a feature age
imputer = imputer.fit(xarray[:,1:4])

In [106]:
xarray[:,1:4] = imputer.transform(xarray[:,1:4])

<h3> Escalonamento dos atributos (Através da padronização) </h3>

In [107]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xarray = scaler.fit_transform(xarray)

<h3> Divisão nas classes de Treino e de Teste </h3>

In [116]:
from sklearn.cross_validation import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(xarray, yarray, test_size = 0.25)

In [117]:
yarray = np.int64(yarray)

<h3> Aplicação do Naive-Bayes </h3>

In [115]:
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()
classificador.fit(xtrain, ytrain)

ValueError: Unknown label type: (array([ 0.        ,  1.        , 40.92770045]),)

In [88]:
previsoes = classificador.predict(xtest)

<h3> Acertou bastante? </h3>

In [91]:
previsoes == ytest

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

<h3> Comparativo para saber a acurácia do modelo </h3>

In [97]:
from sklearn.metrics import confusion_matrix, accuracy_score


In [98]:
precisao = accuracy_score(ytest, previsoes)

ValueError: Classification metrics can't handle a mix of continuous and binary targets